In [19]:
import tensorflow

In [20]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers, optimizers
import PIL
import PIL.Image
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

In [21]:
train=pd.read_csv(r"C:\Users\m1050816\Desktop\ImageRecognitionUseCase\trainLabels.csv",dtype=str)

In [22]:
train.head()

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile


In [23]:
def png_append(fn):
    return fn+".png"

In [24]:
train['id']=train['id'].apply(png_append)

In [25]:
train.head()

,id,label
0,1.png,frog
1,2.png,truck
2,3.png,truck
3,4.png,deer
4,5.png,automobile


In [26]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.20)

In [27]:
dir=r"C:\Users\m1050816\Desktop\ImageRecognitionUseCase\TrainingImages\train\train"

In [28]:
train_generator=datagen.flow_from_dataframe(
dataframe=train,
directory=dir,
x_col='id',
y_col="label",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))

Found 36000 validated image filenames belonging to 10 classes.


In [29]:
valid_generator=datagen.flow_from_dataframe(
dataframe=train,
directory=dir,
x_col="id",
y_col="label",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))

Found 9000 validated image filenames belonging to 10 classes.


In [31]:
IMAGE_SIZE = [224, 224]

In [45]:
vgg = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(32, 32, 3)))

In [46]:
for layer in vgg.layers:
  layer.trainable = False

In [47]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [48]:
flat = Flatten()(vgg.output)
class1 = Dense(1024, activation='relu')(flat)
output = Dense(10, activation='softmax')(class1)

In [49]:
model = Model(inputs=vgg.inputs, outputs=output)

In [50]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [51]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [52]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [ ]:
model.fit_generator(generator=train_generator,steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,validation_steps=STEP_SIZE_VALID,epochs=5
)

Epoch 1/5
 455/1125 [===========>..................] - ETA: 18:23 - loss: 1.4797 - accuracy: 0.4787